In [154]:
import numpy as np
import pandas as pd
import seaborn as sns

In [155]:
# Import my modules.
import sys, os
from pathlib import Path
current_dir = os.path.join(Path().resolve())
sys.path.append(str(current_dir) + '/../')

from modules import utils
from modules import models
from modules import preprocess

import importlib
for m in [utils, models, preprocess]:
    importlib.reload(m)

In [156]:
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)

In [157]:
mbd = 'microbusiness_density'

df_train, df_test, df_subm = utils.load_dataset()
df_all, df_census = utils.merge_dataset(df_train, df_test, pop=False, unemploy=False, census=True, coord=True, fix_pop=True, outlier=False)

/tmp/working/code-analysis/../modules/utils.py:217: DtypeWarning: Columns (102) have mixed types.Specify dtype option on import or set low_memory=False.
  if fix_pop:


In [158]:
params={
        "act_thre": 140,
        "abs_thre": 0,
        "USE_LAG": 7,
        "USE_TREND": False,
        "USE_TREND_MULTI": False,
        "blacklist": [],
        "blacklistcfips": [],
        "clip": (-0.0044, 0.0046),
        "model": 'lgbm'
    }

In [159]:
test_subm = models.LgbmBaseline('test_subm_accum', df_subm, df_all, df_census, start_all_dict=40, params=params)
test_subm.create_submission(save=False)

add lag features: max_scale=40
created df_all_dict[40]
valid_times:  41
pred_m:  1
train_times:  40
use df_all_dict[40]
['scale', 'state_i', 'pct_bb_2017', 'pct_bb_2018', 'pct_bb_2019', 'pct_bb_2020', 'pct_bb_2021', 'pct_college_2017', 'pct_college_2018', 'pct_college_2019', 'pct_college_2020', 'pct_college_2021', 'pct_foreign_born_2017', 'pct_foreign_born_2018', 'pct_foreign_born_2019', 'pct_foreign_born_2020', 'pct_foreign_born_2021', 'pct_it_workers_2017', 'pct_it_workers_2018', 'pct_it_workers_2019', 'pct_it_workers_2020', 'pct_it_workers_2021', 'median_hh_inc_2017', 'median_hh_inc_2018', 'median_hh_inc_2019', 'median_hh_inc_2020', 'median_hh_inc_2021', 'lng', 'lat', 'select_rate1_lag1', 'select_rate1_lag2', 'select_rate1_lag3', 'select_rate1_lag4', 'select_rate1_lag5', 'select_rate1_lag6', 'select_rate1_lag7', 'select_rate1_rsum2', 'select_rate1_rsum4', 'select_rate1_rsum6', 'select_rate1_rsum8', 'select_rate1_rsum10', 'select_active_lag1_diff1', 'select_active_lag1_diff2', 'selec

/opt/conda/lib/python3.7/site-packages/lightgbm/engine.py:177: UserWarning: Found `n_iter` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


# of cfips that have trend multi: 684
saved output_dic[1].
create df_all_dict[41].
add lag features: max_scale=41
success in updating df_all_dict
valid_times:  42
pred_m:  1
train_times:  41
use df_all_dict[41]
use lgb.
[LightGBM] [Warning] lambda_l2 is set=0.5, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.5
[LightGBM] [Warning] min_data_in_leaf is set=213, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=213
[LightGBM] [Warning] seed is set=42, random_state=42 will be ignored. Current value: seed=42
[LightGBM] [Warning] num_iterations is set=30, n_iter=30 will be ignored. Current value: num_iterations=30
# of cfips that have trend multi: 682
saved output_dic[2].
create df_all_dict[42].
add lag features: max_scale=42
success in updating df_all_dict
valid_times:  43
pred_m:  1
train_times:  42
use df_all_dict[42]
use lgb.
[LightGBM] [Warning] lambda_l2 is set=0.5, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.5
[LightGBM] [Warning] min_dat